# Import packages

In [8]:
import pandas as pd
from rdkit import Chem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch_geometric.transforms as T

# Read file and filter data

In [ ]:
chembl_df = pd.read_csv("./data/ChEMBL.csv", sep=";", low_memory=False)
chembl_df.shape

In [ ]:
chembl_df.dtypes

SMILES is needed to get graph representation of a molecule. Thus, missing values should be removed.

In [ ]:
chembl_df["Smiles"].isna().sum()

In [ ]:
chembl_df.dropna(subset=["Smiles"], inplace=True)
chembl_df.shape

In [ ]:
chembl_df["Smiles"].isna().sum()

Drop duplicates.

In [ ]:
chembl_df.drop_duplicates(subset=["Smiles"], inplace=True)
chembl_df.shape

All SMILES that include non-concatenation bonds and cannot be represented as a connected graph, should be removed too.

In [ ]:
chembl_df = chembl_df[~chembl_df["Smiles"].str.contains("\.")]
chembl_df[chembl_df["Smiles"].str.contains("\.")]["Smiles"].sum()

In [ ]:
chembl_df.shape

Remove SMILES with ionized atoms.

In [ ]:
chembl_df = chembl_df[~chembl_df.Smiles.str.contains("\+")]
chembl_df.shape

In [ ]:
chembl_df = chembl_df[~chembl_df.Smiles.str.contains("-]")]
chembl_df.shape

SMILES can be converted into molecules using RDKit package. Hence, SMILES that cannot be converted should be removed too.

In [ ]:
chembl_df["to_mol"] = chembl_df["Smiles"].apply(
    lambda smiles: Chem.MolFromSmiles(smiles) is not None
)
chembl_df[chembl_df["to_mol"]].shape

# Choose data for graph classification

In [ ]:
chembl_df["Molecular Species"].value_counts(dropna=False)

Remove rows that contains None in Molecular Species column.

In [ ]:
chembl_df = chembl_df[chembl_df["Molecular Species"] != "None"]
chembl_df.shape

Choose 100 random records for every Species.

In [ ]:
dataset_df = chembl_df.groupby(by=["Molecular Species"]).sample(n=100, random_state=21)
dataset_df.shape

In [ ]:
dataset_df = dataset_df[["Smiles", "Molecular Species"]]
dataset_df.head()

In [24]:
dataset_df["target"] = LabelEncoder().fit_transform(dataset_df["Molecular Species"])

In [25]:
train_data, test_data = train_test_split(
    dataset_df, stratify=dataset_df["target"], test_size=0.2, random_state=21
)

In [ ]:
train_data.groupby("target").count()

In [ ]:
train_data.to_csv(
    "./data/train_data.csv",
    index=False,
)
test_data.to_csv(
    "./data/test_data.csv",
    index=False,
)

# Construct graphs

In [27]:
def get_one_hot_encoding(x, permitted_list):
    if x not in permitted_list:
        x = permitted_list[-1]

    one_hot_encoded_x = [
        int(is_in_list) for is_in_list in map(lambda s: x == s, permitted_list)
    ]

    return one_hot_encoded_x


def get_node_features(mol):
    permitted_list_of_atoms = [
        "C",
        "N",
        "O",
        "S",
        "F",
        "Si",
        "P",
        "Cl",
        "Br",
        "Mg",
        "Na",
        "Ca",
        "Fe",
        "As",
        "Al",
        "I",
        "B",
        "V",
        "K",
        "Tl",
        "Yb",
        "Sb",
        "Sn",
        "Ag",
        "Pd",
        "Co",
        "Se",
        "Ti",
        "Zn",
        "Li",
        "Ge",
        "Cu",
        "Au",
        "Ni",
        "Cd",
        "In",
        "Mn",
        "Zr",
        "Cr",
        "Pt",
        "Hg",
        "Pb",
        "Unknown",
    ]
    permitted_list_of_hybridization = [
        "S",
        "SP",
        "SP2",
        "SP3",
        "SP3D",
        "SP3D2",
        "OTHER",
    ]
    permitted_list_of_formal_charge = [-3, -2, -1, 0, 1, 2, 3, "Extreme"]

    x = []

    for atom in mol.GetAtoms():
        features = []

        features.extend(get_one_hot_encoding(atom.GetSymbol(), permitted_list_of_atoms))
        features.extend(
            get_one_hot_encoding(
                atom.GetHybridization(), permitted_list_of_hybridization
            )
        )
        features.extend(
            get_one_hot_encoding(
                atom.GetFormalCharge(), permitted_list_of_formal_charge
            )
        )
        features.append(atom.GetIsAromatic())
        features.append(atom.IsInRing())

        x.append(features)

    return torch.tensor(x, dtype=torch.float)


def get_edge_indices(mol):
    start = []
    end = []

    for bond in mol.GetBonds():
        start.append(bond.GetBeginAtomIdx())
        end.append(bond.GetEndAtomIdx())

    return torch.tensor([start, end], dtype=torch.long)


def get_edge_weights(mol):
    edge_weight = []

    for bond in mol.GetBonds():
        edge_weight.append([bond.GetBondTypeAsDouble()])

    return torch.tensor(edge_weight, dtype=torch.float)


def mol_to_pyg_graph(mol, y):
    x = get_node_features(mol)
    edge_index = get_edge_indices(mol)
    edge_weight = get_edge_weights(mol)
    y = torch.tensor([y], dtype=torch.long)

    return T.NormalizeFeatures()(
        T.ToUndirected()(Data(x=x, edge_index=edge_index, edge_weight=edge_weight, y=y))
    )

In [28]:
train_df = pd.read_csv(
    "./data/train_data.csv"
)
test_df = pd.read_csv(
    "./data/test_data.csv"
)

In [29]:
train_data = [
    mol_to_pyg_graph(Chem.MolFromSmiles(smiles), y)
    for smiles, y in train_df[["Smiles", "target"]].values
]
test_data = [
    mol_to_pyg_graph(Chem.MolFromSmiles(smiles), y)
    for smiles, y in test_df[["Smiles", "target"]].values
]

# Create loader for mini-batching

In [32]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Build GCN

In [33]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(60, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 4)

    def forward(self, x, edge_index, edge_weight, batch):
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = self.conv2(x, edge_index, edge_weight)
        x = x.relu()
        x = self.conv3(x, edge_index, edge_weight)

        x = global_mean_pool(x, batch)

        x = torch.nn.functional.dropout(x, p=0.5, training=self.training)

        x = self.lin(x)

        return x

In [ ]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:
        out = model(
            data.x, data.edge_index, data.edge_weight, data.batch
        )
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def test(loader):
    model.eval()

    correct = 0

    for data in loader:
        out = model(data.x, data.edge_index, data.edge_weight, data.batch)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())

    return correct / len(loader.dataset)


for epoch in range(1, 151):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.2f}, Test Acc: {test_acc:.2f}"
    )